# Convert tuples to sentences (tagged with nouns, verbs, and adjectives)

In [522]:
import pandas as pd 
import re
import nltk
from ast import literal_eval

In [729]:
data = pd.read_csv('tagged_lemmatized_reviews.csv') 
df = pd.DataFrame(data, columns=['r_id', 's_id', 'app_name','app_id', 'date', 'title', 'review', 'pos', 'rating'])

In [730]:
tagged_text = df['pos'].tolist()

In [731]:
# Dataframe is converted to list of strings, this code is to remove the strings
for x in range(len(tagged_text)):
    tagged_text[x] = literal_eval(tagged_text[x])

In [732]:
table = {"r_id": [], "s_id": [], "app_name": [], "app_id": [], "date": [], "title": [], "review": [], "pos": [], "tagged_sentences": [], "rating": []}

for x in range(len(df)):
    table["r_id"].append(df['r_id'][x])
    table["s_id"].append(df['s_id'][x])
    table["app_name"].append(df['app_name'][x])
    table["app_id"].append(df['app_id'][x])
    table["date"].append(df['date'][x])
    table["title"].append(df['title'][x])
    table["review"].append(df['review'][x])
    table["pos"].append(df['pos'][x])
    # Convert list of tuples to String
    table["tagged_sentences"].append(' '.join(map(lambda x: str(x[0]), tagged_text[x])))
    table["rating"].append(df['rating'][x])

In [733]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('converted_tuples_to_string.csv', index = False)

# Clean the tagged sentences

In [734]:
data = pd.read_csv('converted_tuples_to_string.csv') 
df = pd.DataFrame(data, columns=['r_id', 's_id', 'app_name','app_id', 'date', 'title', 'review', 'pos', 'tagged_sentences', 'rating'])

In [735]:
len(df)

4118

In [736]:
#Drop empty tagged sentences (those who were included types not nouns, verbs, or adjectives)
df.fillna('', inplace=True)
for x in range(len(df)):
    if not df['tagged_sentences'][x]:
        df.drop(x, inplace=True)

In [737]:
df = df.reset_index(drop=True)

In [738]:
# It was 4118
len(df)

4107

In [739]:
# example = 'lie lie lie' 
# This throws an error because when calculating the bigram measures it throws math error 
# I think it tries to find square root of negative number
df['new_tagged_sentences'] = df['tagged_sentences'].str.replace(r'\b(\w+)(\s+\1)+\b',r'\1')

In [740]:
new_tagged_sentences_list = df['new_tagged_sentences'].tolist()

In [741]:
# Remove cells contain one word
one_word_cells = 0
for x in range(len(df)):
    if not (re.search(' ', new_tagged_sentences_list[x])):
        df.drop(x, inplace=True)
        one_word_cells = one_word_cells + 1
        print(new_tagged_sentences_list[x])
        
print(one_word_cells)

problem
response
baaad
do
thats
ethical
bad
thank
refund
cheat
use
time
order
smh
stay
be
uninstalling
delete
disappointed
uninstall
lie
thank
thank
unacceptable
arrive
stupid
thank
star
stupid
refund
thank
recommend
delete
fresh
app
ontime
love
order
slow
star
order
answer
good
fast
late
hour
thank
thank
do
best
bad
bad
glove
bad
take
bad
best
bad
good
restaurant
bad
switch
bad
bad
io
io
io
ios13
iphone
deliver
expensive
mc
stc
wwf86qd
go
wt5735h
bad
good
goodkeep
be
deduct
gooooooooooood
thank
hand
thank
home
use
disappointing
recommend
cheap
thank
professional
keep
late
order
complicate
love
good
good
credibility
perfect
good
good
best
happen
thank
thank
shame
issue
bye
best
thanks
scam
unreliable
disappointing
thanks
recommend
rude
bad
scam
good
perfect
best
best
bad
bad
lateeee
bad
horrible
thank
good
love
trouble
wood
dollar
thank
thank
be
recommended
be
download
wood
use
thank
download
thank
expensive
recommend
i
love
top
151


In [742]:
df = df.reset_index(drop=True)

In [743]:
len(df)

3956

In [744]:
table = {"r_id": [], "s_id": [], "app_name": [], "app_id": [], "date": [], "title": [], "review": [], "pos": [], "tagged_sentences": [], "rating": [], "new_tagged_sentences": []}
for x in range(len(df)):
    table["r_id"].append(df['r_id'][x])
    table["s_id"].append(df['s_id'][x])
    table["app_name"].append(df['app_name'][x])
    table["app_id"].append(df['app_id'][x])
    table["date"].append(df['date'][x])
    table["title"].append(df['title'][x])
    table["review"].append(df['review'][x])
    table["pos"].append(df['pos'][x])
    table["tagged_sentences"].append(df['tagged_sentences'][x])
    table["rating"].append(df['rating'][x])
    table["new_tagged_sentences"].append(df['new_tagged_sentences'][x])

In [745]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('cleaned_tagged_sentences.csv', index = False)

In [260]:
text = 'take order give delivery time exceed hr minute'
tokens = nltk.wordpunct_tokenize(text)
finder = BigramCollocationFinder.from_words(tokens)
collocation_list = sorted(finder.nbest(bigram_measures.likelihood_ratio, 10))
print(collocation_list)

[('delivery', 'time'), ('exceed', 'hr'), ('give', 'delivery'), ('hr', 'minute'), ('order', 'give'), ('take', 'order'), ('time', 'exceed')]


# Find collocations

In [477]:
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures
import itertools

In [746]:
data = pd.read_csv('cleaned_tagged_sentences.csv') 
df = pd.DataFrame(data, columns=['r_id', 's_id', 'app_name','app_id', 'date', 'title', 'review', 'pos', 'tagged_sentences', 'new_tagged_sentences', 'rating'])

In [747]:
new_tagged_sentences_list = df['new_tagged_sentences'].tolist()

In [748]:
# Tokenize each row in the new_tagged_sentences_list
tokens = []
for i in new_tagged_sentences_list:
    j = i.split()
    for p in j:
        p = p.lstrip(' ')
        p = p.rstrip(' ')
        flag = 1
        if(len(p)<2):
            flag = 0
        if(flag):
            tokens.append(p)

In [754]:
# Finding collocations
bigram_measures = nltk.collocations.BigramAssocMeasures()

# reviews = ' '.join(list(itertools.chain.from_iterable(df['new_tagged_sentences'])))
# tokens = nltk.word_tokenize(reviews)
finder = BigramCollocationFinder.from_words(tokens, window_size = 2)

# Keep collocations appear more than 3 times
finder.apply_freq_filter(3)
collocation_list = finder.nbest(bigram_measures.likelihood_ratio, 2000)
print("The top 100 collocations: ", collocation_list)
# print(finder.nbest(bigram_measures.likelihood_ratio, 100))
how_many = 0
for i in finder.score_ngrams(bigram_measures.likelihood_ratio):
    how_many = how_many + 1
    print(i)
# collocation_list = sorted(finder.nbest(bigram_measures.likelihood_ratio, 10))

The top 100 collocations:  [('customer', 'service'), ('food', 'delivery'), ('use', 'app'), ('be', 'be'), ('apple', 'pay'), ('cancel', 'order'), ('easy', 'use'), ('take', 'money'), ('delivery', 'app'), ('customer', 'support'), ('more', 'hour'), ('long', 'time'), ('credit', 'card'), ('do', 'recommend'), ('wait', 'hour'), ('bad', 'experience'), ('app', 'be'), ('take', 'more'), ('take', 'long'), ('payment', 'method'), ('fast', 'delivery'), ('be', 'order'), ('payment', 'option'), ('other', 'apps'), ('call', 'center'), ('download', 'app'), ('support', 'team'), ('order', 'food'), ('delete', 'app'), ('customer', 'care'), ('quality', 'food'), ('be', 'app'), ('place', 'order'), ('bad', 'application'), ('costumer', 'service'), ('delivery', 'fee'), ('first', 'time'), ('do', 'receive'), ('food', 'online'), ('be', 'bad'), ('be', 'easy'), ('bad', 'service'), ('affordable', 'price'), ('arabic', 'english'), ('bad', 'app'), ('accept', 'order'), ('miss', 'item'), ('many', 'time'), ('waste', 'time'), ('kn

(('service', 'order'), 5.884676265414111)
(('fix', 'app'), 5.881463081648983)
(('way', 'get'), 5.867947803917453)
(('few', 'time'), 5.860844405211749)
(('take', 'minute'), 5.857632787892798)
(('cuisine', 'app'), 5.8470158681250695)
(('improve', 'app'), 5.8470158681250695)
(('do', 'reach'), 5.835350215595101)
(('change', 'get'), 5.825069389180976)
(('option', 'use'), 5.747788002051789)
(('think', 'be'), 5.7464998285991555)
(('app', 'need'), 5.731099732705152)
(('ask', 'money'), 5.722794754000874)
(('anything', 'do'), 5.675861882233295)
(('cost', 'be'), 5.674787804508343)
(('finish', 'be'), 5.674787804508343)
(('online', 'get'), 5.652277364609297)
(('app', 'show'), 5.629373972060161)
(('like', 'app'), 5.629373972060161)
(('message', 'customer'), 5.602017953895232)
(('easy', 'refund'), 5.564802646100417)
(('help', 'application'), 5.533990183847104)
(('problem', 'make'), 5.5011191917329745)
(('customer', 'have'), 5.49848158069978)
(('such', 'bad'), 5.482694146319278)
(('process', 'be'), 5.

In [891]:
# Finding collocations
bigram_measures = nltk.collocations.BigramAssocMeasures()

# reviews = ' '.join(list(itertools.chain.from_iterable(df['new_tagged_sentences'])))
# tokens = nltk.word_tokenize(reviews)
finder = BigramCollocationFinder.from_words(tokens, window_size = 2)

# Keep collocations appear more than 3 times
finder.apply_freq_filter(3)
collocation_list = finder.nbest(bigram_measures.pmi, 2000)
print("The top 100 collocations: ", collocation_list)
# print(finder.nbest(bigram_measures.likelihood_ratio, 100))
how_many = 0
for i in finder.score_ngrams(bigram_measures.pmi):
    how_many = how_many + 1
    print(i)
# collocation_list = sorted(finder.nbest(bigram_measures.likelihood_ratio, 10))

The top 100 collocations:  [('quite', 'unsure'), ('abundant', 'fastfood'), ('indian', 'chinese'), ('italian', 'indian'), ('uber', 'eats'), ('knowledgeable', 'educate'), ('restauratns', 'offering'), ('gratify', 'tastebud'), ('lame', 'excuse'), ('wan', 'na'), ('avail', 'excite'), ('positive', 'feeling'), ('proactive', 'patient'), ('executive', 'attend'), ('surprise', 'visit'), ('attend', 'listen'), ('onestopsolution', 'satisfy'), ('little', 'bit'), ('offering', 'clean'), ('superreliable', 'operation'), ('patient', 'listen'), ('google', 'map'), ('top', 'elite'), ('stand', 'door'), ('access', 'photo'), ('perfection', 'lover'), ('satisfied', 'software'), ('delicacy', 'search'), ('satisfy', 'tastebud'), ('tantalize', 'taste'), ('taste', 'bud'), ('hygiene', 'perfection'), ('ocd', 'hygiene'), ('ters', 'hygiene'), ('follow', 'legacy'), ('visit', 'friend'), ('listen', 'al'), ('vast', 'range'), ('hunger', 'pang'), ('specific', 'type'), ('status', 'past'), ('system', 'superreliable'), ('eat', 'ita

In [687]:
collocation_list[1][0] + ' ' + collocation_list[1][1]

'apple pay'

In [751]:
how_many

2010

In [755]:
# Convert collocations pair to string
collocation_sent = []
for x in range(len(collocation_list)):
    phrase = collocation_list[x][0] + ' ' + collocation_list[x][1]
    print(phrase)
    collocation_sent.insert(x, phrase)

customer service
food delivery
use app
be be
apple pay
cancel order
easy use
take money
delivery app
customer support
more hour
long time
credit card
do recommend
wait hour
bad experience
app be
take more
take long
payment method
fast delivery
be order
payment option
other apps
call center
download app
support team
order food
delete app
customer care
quality food
be app
place order
bad application
costumer service
delivery fee
first time
do receive
food online
be bad
be easy
bad service
affordable price
arabic english
bad app
accept order
miss item
many time
waste time
knowledgeable educate
be good
speak arabic
compare other
delightful meal
tasty food
last time
do know
order order
try contact
delivery guy
specific type
free delivery
google map
recommend anyone
be food
contact customer
be able
wide variety
steal money
give star
favorite food
process delivers
receive order
be busy
different cuisine
bad customer
respect customer
be best
vast range
satisfied software
delivery apps
app have

application say
app much
be pay
problem driver
be month
service offer
time change
chef have
application easy
experience application
enjoy order
bad do
app customer
delivery easy
delivery experience
food option
app affordable
know get
order everything
be improve
be notify
be wide
delivery minute
people have
app come
love service
good customer
food quality
order deliver
restaurant recommend
service reply
thing app
time day
refund bad
bring order
location do
driver take
app excellent
choose food
problem delivery
app enjoy
notification app
do nothing
minute order
order wrong
meal app
payment get
money say
app driver
be write
payment be
time customer
see do
get delivery
have deal
help customer
food do
food order
item do
be one
door be
error be
offer be
order want
do take
app bring
app eat
have reach
have find
restaurant food
use service
be amazing
choice be
be contact
make customer
app support
support app
recommend be
do provide
be respond
guy be
app good
order several
pay driver
support ma

In [756]:
collocation_sent

['customer service',
 'food delivery',
 'use app',
 'be be',
 'apple pay',
 'cancel order',
 'easy use',
 'take money',
 'delivery app',
 'customer support',
 'more hour',
 'long time',
 'credit card',
 'do recommend',
 'wait hour',
 'bad experience',
 'app be',
 'take more',
 'take long',
 'payment method',
 'fast delivery',
 'be order',
 'payment option',
 'other apps',
 'call center',
 'download app',
 'support team',
 'order food',
 'delete app',
 'customer care',
 'quality food',
 'be app',
 'place order',
 'bad application',
 'costumer service',
 'delivery fee',
 'first time',
 'do receive',
 'food online',
 'be bad',
 'be easy',
 'bad service',
 'affordable price',
 'arabic english',
 'bad app',
 'accept order',
 'miss item',
 'many time',
 'waste time',
 'knowledgeable educate',
 'be good',
 'speak arabic',
 'compare other',
 'delightful meal',
 'tasty food',
 'last time',
 'do know',
 'order order',
 'try contact',
 'delivery guy',
 'specific type',
 'free delivery',
 'google 

# Tag the extracted features

In [692]:
# Didn't use this
from nltk import word_tokenize, pos_tag, pos_tag_sents

tagged_collocation = pos_tag_sents(map(word_tokenize, collocation_sent))

In [696]:
# Convert NN|NNS|NNPS to n (noun) & VB|VBD|VBG|VBN|VBP|VBZ to v (verb) & JJ|JJR|JJS to a (adjective)
nouns = 0
adj = 0
verbs = 0
for x in range(len(tagged_collocation)):
    for y in range(len(tagged_collocation[x])):
        token, tag = tagged_collocation[x][y]
        if re.match(r'NN|NNS|NNPS', tag): 
            tagged_collocation[x][y] = (token, 'n')
            nouns = nouns + 1
        elif re.match(r'JJ|JJR|JJS', tag):
            tagged_collocation[x][y] = (token, 'a')
            adj = adj + 1
        elif re.match(r'VB|VBD|VBG|VBN|VBP|VBZ', tag):
            tagged_collocation[x][y] = (token, 'v')
            verbs = verbs + 1

print("Number of nouns:", nouns)
print("Number of adjectives:", adj)
print("Number of verbs:", verbs)

Number of nouns: 118
Number of adjectives: 44
Number of verbs: 32


In [710]:
# Check the collocations pattern
new_list = []
for x in range(len(tagged_collocation)):
    token, tag = tagged_collocation[x][1]
    if re.match(r'n', tag):
        print("match word", token, tag)
        new_list.insert(x, tagged_collocation[x])
    else:
        print("Unwanted tage", tag)

match word delivery n
match word pay n
Unwanted tage v
match word app n
match word use n
match word order n
match word money n
match word app n
match word hour n
match word card n
match word service n
match word time n
match word team n
match word hour n
match word service n
Unwanted tage a
Unwanted tage a
match word option n
Unwanted tage v
match word application n
match word delivery n
match word food n
Unwanted tage v
match word order n
match word experience n
match word app n
match word app n
Unwanted tage v
match word time n
Unwanted tage a
match word east n
match word method n
Unwanted tage a
match word food n
match word center n
match word time n
match word fee n
Unwanted tage a
match word educate n
Unwanted tage a
match word english n
match word item n
match word time n
match word middle n
match word money n
match word feature n
match word order n
match word order n
match word delivery n
match word food n
match word customer n
match word apps n
match word software n
match word 

In [723]:
# Convert the collocations following the pattern into sentences
collocations_follow_pattern = []
for x in range(len(new_list)):
    phrase = new_list[x][0][0] + ' ' + new_list[x][1][0] 
    print(phrase)
    collocations_follow_pattern.insert(x, phrase)

food delivery
apple pay
use app
easy use
cancel order
take money
delivery app
more hour
credit card
bad service
long time
support team
wait hour
costumer service
payment option
bad application
fast delivery
order food
be order
bad experience
delete app
bad app
first time
middle east
payment method
quality food
call center
many time
delivery fee
knowledgeable educate
arabic english
miss item
waste time
platform middle
steal money
track feature
accept order
place order
free delivery
be food
respect customer
delivery apps
satisfied software
give star
do care
vast range
favorite food
delivery guy
speak arabic
last time
best food
bookmark favorite
pay apple
technical support
fall love
receive order
recommend anyone
take hour
deliver doorstep
best part
flexible payment
attend listen
follow legacy
best qulaity
last month
refund money
change language
update relate
deliver order
try contact
least week
late delivery
software people
food choice
executive attend
indian chinese
cook food
have probl

In [758]:
reviews_list = df['review'].tolist()

In [ ]:
# Till here ^

# Find sentences of the meaningful features

In [842]:
data1 = pd.read_csv('reviews_without_Arabic_sentences.csv') 
df1 = pd.DataFrame(data1, columns=['r_id', 's_id','app_name','app_id', 'date', 'title', 'review', 'rating'])

In [843]:
# Remove spaces before and after a sentence
df1['review'] = df1['review'].str.strip()

In [844]:
# Because the reviews from the file splitted_reviews_into_sentences_sID are splitted into sentences, and the Arabic sentences are removed
sentences_list = df1['review'].tolist()

In [845]:
data2 = pd.read_csv('extracted_features.csv') 
df2 = pd.DataFrame(data2, columns=['extracted_features'])

In [846]:
df2.fillna('', inplace=True)
for x in range(len(df2)):
    if not df2['extracted_features'][x]:
        df2.drop(x, inplace=True)

In [847]:
df2 = df2.reset_index(drop=True)

In [848]:
featres_list = df2['extracted_features'].tolist()

In [849]:
featres_list

['customer service',
 'food delivery',
 'use app',
 'be be',
 'apple pay',
 'cancel order',
 'easy use',
 'take money',
 'delivery app',
 'customer support',
 'more hour',
 'long time',
 'credit card',
 'do recommend',
 'wait hour',
 'bad experience',
 'app be',
 'take more',
 'take long',
 'payment method',
 'fast delivery',
 'be order',
 'payment option',
 'other apps',
 'call center',
 'download app',
 'support team',
 'order food',
 'delete app',
 'customer care',
 'quality food',
 'be app',
 'place order',
 'bad application',
 'costumer service',
 'delivery fee',
 'first time',
 'do receive',
 'food online',
 'be bad',
 'be easy',
 'bad service',
 'affordable price',
 'arabic english',
 'bad app',
 'accept order',
 'miss item',
 'many time',
 'waste time',
 'knowledgeable educate',
 'be good',
 'speak arabic',
 'compare other',
 'delightful meal',
 'tasty food',
 'last time',
 'do know',
 'order order',
 'try contact',
 'delivery guy',
 'specific type',
 'free delivery',
 'google 

In [850]:
# Find each features with its reference review/sentence
table = {"r_id": [], "s_id": [], "reference_sentence": []}
counter = 0

for x in range(len(featres_list)):
    print("-------feature", featres_list[x])
    for y in range(len(sentences_list)):
#         Same results if I use re.search
        if featres_list[x] in sentences_list[y].lower():
            counter = counter + 1
            print("Feature:", featres_list[x])
            print("Found in row #:", y, "|| Sentence::", sentences_list[y])
            table["r_id"].append(df1['r_id'][y])
            table["s_id"].append(df1['s_id'][y])
            table["reference_sentence"].append(sentences_list[y])
        

print(counter)

-------feature customer service
Feature: customer service
Found in row #: 31 || Sentence:: There is no customer service number, you get los whenever you have issue .
Feature: customer service
Found in row #: 37 || Sentence:: Bcs i sent my complaint and there's no response from the customer service.
Feature: customer service
Found in row #: 42 || Sentence:: Surprisingly the customer service only told me that they will call me later to return the money.
Feature: customer service
Found in row #: 54 || Sentence:: Very limited and complicated application, extremely poor delivery service and non-existing customer service.
Feature: customer service
Found in row #: 55 || Sentence:: .One order out of 3 would reach me missing items and I'd be forced to sit through the chat with their customer service but not once was my problem solved or my money reimbursed.
Feature: customer service
Found in row #: 61 || Sentence:: I think the customer in this case need to communicate customer service which is 

Found in row #: 2416 || Sentence:: The customer service is so bad I have a situation to cancel the order and I communicate with the Twitter account they told me with my attitude that I can't cancel the order and a if I am not going to receive the order it will Religion on me
Feature: customer service
Found in row #: 2424 || Sentence:: Best company.Best service .Best customer service .Thank you all!!!
Feature: customer service
Found in row #: 2469 || Sentence:: Integrate a better customer service platform.
Feature: customer service
Found in row #: 2482 || Sentence:: Order is not complete and customer service are a the worst ..They took my money and i didn't receive my order
Feature: customer service
Found in row #: 2501 || Sentence:: They updated my request as delivered and didn't refund anything!.Very bad App and customer service
Feature: customer service
Found in row #: 2543 || Sentence:: Very poor customer service and management
Feature: customer service
Found in row #: 2569 || Sente

Feature: use app
Found in row #: 3703 || Sentence:: ‍‍​ Quite nice service with fast delivery and easy to use app.
Feature: use app
Found in row #: 3798 || Sentence:: Quite nice service with fast delivery and easy to use app.
Feature: use app
Found in row #: 3846 || Sentence:: Quite nice service with fast delivery and easy to use app.
Feature: use app
Found in row #: 5186 || Sentence:: Easy to use app.
-------feature be be
Feature: be be
Found in row #: 559 || Sentence:: With all updates it be better and better
Feature: be be
Found in row #: 803 || Sentence:: You would be better off with ur money ( after you place an order they don't care if it wrong or something is missing) very bad for long term busses
-------feature apple pay
Feature: apple pay
Found in row #: 33 || Sentence:: I ordered on the 16th of June and the payment was done by Apple Pay.
Feature: apple pay
Found in row #: 38 || Sentence:: I think it's fine if i share my unpleasant experience yesterday..I was order some foods 

Found in row #: 4549 || Sentence:: I consider it to be the easiest way to order food and I loved it so much that I recommend this great food delivery app to be used by anyone when ordering food.
Feature: delivery app
Found in row #: 4558 || Sentence:: It is without, any doubt, the best food delivery app.
Feature: delivery app
Found in row #: 4566 || Sentence:: ‌  ‌Best food delivery app to download and there's no commitment to use most of all.
Feature: delivery app
Found in row #: 4580 || Sentence:: Undoubtedly Chefz is the best online food delivery app in Abu Dhabi.‍  ‍
Feature: delivery app
Found in row #: 4616 || Sentence:: Experience the lightening fast delivery app service across the UAE.
Feature: delivery app
Found in row #: 4621 || Sentence:: It's a fantastic food delivery app.
Feature: delivery app
Found in row #: 4644 || Sentence:: Unlike any other food delivery apps, The Chef offers the food within the given time-frame.
Feature: delivery app
Found in row #: 4650 || Sentence::

Feature: credit card
Found in row #: 4444 || Sentence:: Most of the time I pay for the food using my credit card.
Feature: credit card
Found in row #: 4816 || Sentence:: I tried adding two different credit cards and it says "Card number invalid" or something like that.
-------feature do recommend
-------feature wait hour
-------feature bad experience
Feature: bad experience
Found in row #: 253 || Sentence:: They can't manage their drivers, the driver take long time to deliver my order, I received the food very cold because the driver is far away from the restaurant and my location, I think he take another order which is not acceptable, the customer service response is simply (sorry), this is the 2nd bad experience with this application although there is another applications better than this.
Feature: bad experience
Found in row #: 714 || Sentence:: No support from customer service when you face a problem or bad experience from orders and restaurant ..They just till you we are sorry.
Fe

...

Feature: delightful meal
Found in row #: 5077 || Sentence:: It brings delightful meals and the lightning fast delivery of the food makes the app best in the food delivery process.
Feature: delightful meal
Found in row #: 5134 || Sentence:: It brings delightful meals and the lightning fast delivery of the food makes the app best in the food delivery process.
Feature: delightful meal
Found in row #: 5265 || Sentence:: Enjoy delightful meals from The Chefz app and avail exciting discounts on almost all orders with contactless delivery.
Feature: delightful meal
Found in row #: 5278 || Sentence:: Enjoy delightful meals from The Chefz app and avail exciting discounts on almost all orders with contactless delivery.
Feature: delightful meal
Found in row #: 5295 || Sentence:: ‍  Enjoy delightful meals from The Chefz app and avail exciting discounts on almost all orders with contactless delivery.
-------feature tasty food
Feature: tasty food
Found in row #: 3706 || Sentence:: I am a little bit d

Found in row #: 1237 || Sentence:: Worst customer service, I don't recommend anyone to download this app.
Feature: recommend anyone
Found in row #: 1274 || Sentence:: ..I don't recommend anyone to deal with HS.
Feature: recommend anyone
Found in row #: 2363 || Sentence:: My first time to use app was  27 March and I do not recommend anyone to use ..I live in KSA , Riaydh and we are in Curfew and I have use their app at 1.20 pm which is before curfew with almost one hour hand half..They accept the order and they confirmed that store accept it as well and till time of curfew now one respond no one explained and it's end up that me and my family has no lunch ..If they just being responsive and aware of situation and deny my order I would have gone and order myself from any nearby store ..Bad do not try even
Feature: recommend anyone
Found in row #: 3281 || Sentence:: I don't recommend anyone to download the app , trying to reach them for almost 1 week to retrieve my money and no answer at 

-------feature vast range
Feature: vast range
Found in row #: 3705 || Sentence:: Restaurant selection is very wide so I could really decide what I wanted from a vast range.
Feature: vast range
Found in row #: 3800 || Sentence:: Restaurant selection is very wide so I could really decide what I wanted from a vast range.
Feature: vast range
Found in row #: 3848 || Sentence:: Restaurant selection is very wide so I could really decide what I wanted from a vast range.‌ ‌
Feature: vast range
Found in row #: 4459 || Sentence:: ‌ Now you can choose any resturant as per your choice and order food from a vast range of menus.
Feature: vast range
Found in row #: 4496 || Sentence:: Now you can choose any resturant as per your choice and order food from a vast range of menus.
-------feature satisfied software
-------feature delivery apps
Feature: delivery apps
Found in row #: 236 || Sentence:: Use other delivery apps to avoid such incidents.
Feature: delivery apps
Found in row #: 295 || Sentence:: .H

Found in row #: 4901 || Sentence:: I use this app for ordering best quality chocolates from the stores in Riyadh.
Feature: best quality
Found in row #: 4902 || Sentence:: I love chocolates and they offer best quality chocolates.
Feature: best quality
Found in row #: 4964 || Sentence:: I use this app for ordering best quality chocolates from the stores in Riyadh.
Feature: best quality
Found in row #: 4965 || Sentence:: I love chocolates and they offer best quality chocolates.
-------feature people produce
-------feature food lover
Feature: food lover
Found in row #: 3646 || Sentence:: ‍If you are a food lover and love to browse dishes information to try something new like me, then download the Chefz app.
Feature: food lover
Found in row #: 3662 || Sentence:: ‌If you are a food lover and love to browse dishes information to try something new like me, then download the Chefz app.
Feature: food lover
Found in row #: 3676 || Sentence:: If you are a food lover and love to browse dishes infor

-------feature deliver order
-------feature quite unsure
Feature: quite unsure
Found in row #: 3987 || Sentence:: ‍ When I first downloaded the chefz app I was quite unsure, but when I used this app, I got to know it's interesting features that make it unique from others.
Feature: quite unsure
Found in row #: 4190 || Sentence:: When I first downloaded the chefz app I was quite unsure, but when I used this app, I got to know it's interesting features that make it unique from others.
Feature: quite unsure
Found in row #: 4359 || Sentence:: When I first downloaded the chefz app I was quite unsure, but when I used this app, I got to know it's interesting features that make it unique from others.
-------feature take hour
Feature: take hour
Found in row #: 1018 || Sentence:: Orders take hours to be delivered and many times they reject orders after 1 hour.
-------feature flexible payment
Feature: flexible payment
Found in row #: 4085 || Sentence:: ​  What makes theapp best among all its compe

In [853]:
# This file will be used in the sentiment analysis tool
df=pd.DataFrame.from_dict(table,orient='index').transpose()
# Remove duplicate reference sentences
df = df[['r_id', 's_id', 'reference_sentence']].drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
df = df.reset_index(drop=True)

In [854]:
df

,r_id,s_id,reference_sentence
0,3,31,"There is no customer service number, you get l..."
1,5,37,Bcs i sent my complaint and there's no respons...
2,5,42,Surprisingly the customer service only told me...
3,7,58,"Very limited and complicated application, extr..."
4,7,59,.One order out of 3 would reach me missing ite...
...,...,...,...
1160,10467,13982,Till now I have never experienced a late deliv...
1161,342,899,"Stupid app , i always have problem with discou..."
1162,10132,13009,‍ When I first downloaded the chefz app I was ...
1163,10196,13212,When I first downloaded the chefz app I was qu...


In [855]:
df.to_csv('reference_sentences.csv', index = False)

# Find reviews of the referenced sentences

In [856]:
data1 = pd.read_csv('reviews_without_Arabic_reviews_sentences.csv') 
df1 = pd.DataFrame(data1, columns=['r_id', 's_id','app_name','app_id', 'date', 'title', 'review', 'rating'])

In [881]:
reviews_id_from_reviews = df1['r_id'].tolist()
reviews_list = df1['review'].tolist()

In [867]:
reviews_id_from_reviews

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [857]:
data2 = pd.read_csv('reference_sentences.csv') 
df2 = pd.DataFrame(data2, columns=['r_id', 's_id','reference_sentence'])

In [880]:
sentences_list = df2['reference_sentence'].tolist()
print(sentences_list)

['There is no customer service number, you get los whenever you have issue .', "Bcs i sent my complaint and there's no response from the customer service.", 'Surprisingly the customer service only told me that they will call me later to return the money.', 'Very limited and complicated application, extremely poor delivery service and non-existing customer service.', ".One order out of 3 would reach me missing items and I'd be forced to sit through the chat with their customer service but not once was my problem solved or my money reimbursed.", 'I think the customer in this case need to communicate customer service which is time and energy consuming for the customer to get his money back for this day to day encounters.', "..The driver also informed me that HungerStation instructed him to mark the order as delivered and to leave my order at the location shown on the faulty app !..I will delete this app now since the customer service did not try to correct the issue or at least  refund th

In [860]:
reviews_id_from_sentences = df2['r_id'].tolist()

In [868]:
reviews_id_from_sentences

[3,
 5,
 5,
 7,
 7,
 8,
 10,
 12,
 12,
 16,
 16,
 16,
 18,
 21,
 21,
 22,
 26,
 28,
 29,
 31,
 31,
 43,
 45,
 46,
 46,
 49,
 55,
 60,
 63,
 63,
 68,
 99,
 106,
 113,
 124,
 135,
 144,
 153,
 156,
 158,
 164,
 173,
 174,
 175,
 181,
 189,
 215,
 219,
 220,
 222,
 223,
 245,
 248,
 253,
 264,
 277,
 338,
 344,
 365,
 370,
 378,
 399,
 406,
 414,
 415,
 416,
 423,
 425,
 437,
 468,
 470,
 477,
 480,
 481,
 484,
 499,
 508,
 514,
 519,
 523,
 543,
 557,
 558,
 572,
 574,
 596,
 621,
 646,
 650,
 669,
 682,
 692,
 707,
 710,
 711,
 720,
 768,
 4938,
 6058,
 6069,
 6072,
 6115,
 6119,
 6120,
 6121,
 6124,
 6128,
 6136,
 6148,
 6157,
 6166,
 6177,
 6182,
 6186,
 6202,
 6206,
 6217,
 6222,
 6251,
 6264,
 6278,
 6309,
 6334,
 6365,
 6370,
 6390,
 8060,
 8068,
 8073,
 8073,
 8077,
 8083,
 8091,
 8091,
 8102,
 8102,
 8103,
 8106,
 8117,
 8120,
 8125,
 8129,
 8132,
 8136,
 8139,
 8141,
 8142,
 8148,
 8149,
 8151,
 8162,
 8168,
 8173,
 8175,
 8188,
 8192,
 8196,
 8202,
 8210,
 8212,
 8220,
 8224,
 

In [885]:
# Find original review based on r_id
table = {"r_id": [], "reference_review": []}
counter = 0
for x in range(len(reviews_id_from_sentences)):
    for y in range(len(reviews_id_from_reviews)):
        if reviews_id_from_sentences[x] == reviews_id_from_reviews[y]:
            print("Sentence r_id: ", reviews_id_from_sentences[x], "equals review r_id: ", reviews_id_from_reviews[y])
            print("*Sentence", sentences_list[x])
            print("**Review", reviews_list[y])
            print("-------")
            counter = counter + 1
            table["r_id"].append(reviews_id_from_reviews[y])
            table["reference_review"].append(reviews_list[y])
        

print("Reference reviews: ", counter)

Sentence r_id:  3 equals review r_id:  3
*Sentence There is no customer service number, you get los whenever you have issue .
**Review This app has many issues, technical and personal as well , first of all never deliver on time, never stick to given promised time. Regarding the personal , they always come to you directly on whats app asking to share location, knowing the app has your full details this is uncomfortable imagine the amount unknown numbers you will hVe . There are times where they don't deliver the order at all, then you have to chase them in twitter otherwise they do not refund your money back , staff attitude is low, they deal with you in not friendly way, weak in arabic and english as well, no respect in dealing  , besides if you live in apartment they refuse to bring it up . Delivery cost is high knowing the app shows only near by restaurant and  still high . There is no customer service number, you get los whenever you have issue . .Honestly one of the worst experien

Sentence r_id:  543 equals review r_id:  543
*Sentence Worst customer service ever.
**Review Worst customer service ever. They finished the conversation while I was talking to them.
-------
Sentence r_id:  557 equals review r_id:  557
*Sentence Their Customer Service one of our the worst in KSA
**Review Their Customer Service one of our the worst in KSA
-------
Sentence r_id:  558 equals review r_id:  558
*Sentence Too expensive and worst customer service.غالي و اسوء خدمة عملاء ماتقدر تتواصل معهم
**Review Too expensive and worst customer service.غالي و اسوء خدمة عملاء ماتقدر تتواصل معهم
-------
Sentence r_id:  572 equals review r_id:  572
*Sentence Very Poor customer service
**Review Very Poor customer service
-------
Sentence r_id:  574 equals review r_id:  574
*Sentence Worst customer service, I don't recommend anyone to download this app.
**Review Worst customer service, I don't recommend anyone to download this app.
-------
Sentence r_id:  596 equals review r_id:  596
*Sentence if 

Sentence r_id:  10637 equals review r_id:  10637
*Sentence 5 stars to their customer service.
**Review 5 stars to their customer service. Literally one of the top. .Lovely app with variety of choices and great prices.
-------
Sentence r_id:  10815 equals review r_id:  10815
*Sentence .They have no customer services, and their services are not customer centered..My last 5 digits 97401.One more addition, I picked up my last order and they deducted and charged me for the delivery
**Review The app is excellent, however, if you have any problems with the payments or the delivery you cannot contact the hotline if they have any. Your payment will go and you will not receive any food if the driver decided not to take the order, you will have to go and pick it up your self or make a new order from another app. My recommendation is to find another app unless you don't have a problem going your self or waiting for them until they feel like delivering your order. .They have no customer services, a

Sentence r_id:  10197 equals review r_id:  10197
*Sentence Some of the best app for food delivery.
**Review     Some of the best app for food delivery. This app shows my new order status and My past orders in case I ordered something and I like it and forget the name of the meal it helps a lot.‍    ‌‌
-------
Sentence r_id:  10203 equals review r_id:  10203
*Sentence Really helpful these days, user friendly interface as well, there is no need for other food delivery apps once it has started using.
**Review    Really helpful these days, user friendly interface as well, there is no need for other food delivery apps once it has started using. ‌‍‌‌
-------
Sentence r_id:  10212 equals review r_id:  10212
*Sentence The chefz is the best food delivery app in the United Arab Emirate.
**Review ‌ ‌ The app lets you know about the estimated delivery time so you don't have to stand up on the door waiting for the person. The chefz is the best food delivery app in the United Arab Emirate. ​
-------

Sentence r_id:  10564 equals review r_id:  10564
*Sentence ‌This is a really good food delivery app.
**Review    ‌This is a really good food delivery app. The app is very user friendly. It delivers meals on time. There's no glitch in the app and is amazing. ‌
-------
Sentence r_id:  10578 equals review r_id:  10578
*Sentence ‍ The Chefz app is best food delivery app in UAE.
**Review  ‍ The Chefz app is best food delivery app in UAE. You get to browse through different cuisines. It is the best restaurant finder in UAE. Must download the Chefz app. ​    ​
-------
Sentence r_id:  10586 equals review r_id:  10586
*Sentence ‌ Excellent food delivery app, and operation.
**Review ‌ Excellent food delivery app, and operation. Delivery workers are respectful and well behaved .And 5 stars for the precautions during COVID-19     ​
-------
Sentence r_id:  10589 equals review r_id:  10589
*Sentence Superb food delivery app with great service.
**Review   Superb food delivery app with great service. 

...

Sentence r_id:  10340 equals review r_id:  10340
*Sentence Grab this app and order food of your choice.
**Review   If you are planning to order food in bulk, then without any wait. Grab this app and order food of your choice. The Chefz is the best app for ordering bulk ordering. ‍
-------
Sentence r_id:  10342 equals review r_id:  10342
*Sentence Whenever I feel cravings for good food, I order food using the chefz app.
**Review    Whenever I feel cravings for good food, I order food using the chefz app. You get the food from the convenience of your home and at affordable price tags. ‌‍‍
-------
Sentence r_id:  10358 equals review r_id:  10358
*Sentence Gratify your taste-buds and hunger calls order food online on The Chefz.
**Review    Gratify your taste-buds and hunger calls order food online on The Chefz. You can find every food option on the chefz app at affordable prices. ​  ​
-------
Sentence r_id:  10373 equals review r_id:  10373
*Sentence I can browse the restaurants and order 

Sentence r_id:  6340 equals review r_id:  6340
*Sentence Your delivery fees are really high
**Review Your delivery fees are really high
-------
Sentence r_id:  8070 equals review r_id:  8070
*Sentence I know you can return it to your stall and sell it again so both of you gain the money you didnt lose anything because you even take delivery fee .
**Review They will never refund you, and not the mistake of the consumer if the google map is wrong. And not the mistake of the consumer if the shop cannot deliver the order. And the shop just simply tell the consumer to cancel it. To you and the shop should treat the consumer fair you take our money without anything it's like a "thief" .. I'm not angry because the money was not refunded I am angry because you didnt do anything for the consumer in just one click you gain the money. Really it's really harami. I think you better check your policy again. Because what if it will happen to other consumer because anytime you can tell anyone to cance

Sentence r_id:  267 equals review r_id:  267
*Sentence I've reported so many times but it seems to be ignored for not having real integrity
**Review I've been charged so many times via my visa and never got my order, nor my money back. I've reported so many times but it seems to be ignored for not having real integrity
-------
Sentence r_id:  288 equals review r_id:  288
*Sentence And other staff just for chit chat very experience with them more than many time goint to delete this from my phone
**Review Their drivers dont know even how to use mobile apps. And other staff just for chit chat very experience with them more than many time goint to delete this from my phone
-------
Sentence r_id:  341 equals review r_id:  341
*Sentence Many times the driver assigned and order not delivered and they simply cancel your order after waiting more than one hour.
**Review Many times the driver assigned and order not delivered and they simply cancel your order after waiting more than one hour. Stup

-------
Sentence r_id:  6226 equals review r_id:  6226
*Sentence You have to select your driver very carful .Need to be able to communicate with good behavior and dressed well.
**Review You have to select your driver very carful .Need to be able to communicate with good behavior and dressed well.
-------
Sentence r_id:  8069 equals review r_id:  8069
*Sentence I had to go and apologize to all the other 16 people that I will not be able arrange the food today as all the other places in my locality was closed due to the lockdown or some other reasons.
**Review Worst app ever! Never use this app. I ordered food for 17 people on May 22nd 2020. The order was scheduled for May 23rd 2020, ie I made the order 24hrs before. I know it's Ramadan season and there is rush but you should also have the ethics to inform your customer that you cannot deliver. If you do not have the workforce or server capacity then why did you take my money and kept mum. You will refund my money, but what about the ins

Sentence r_id:  10214 equals review r_id:  10214
*Sentence It is the finest restaurant finder app in UAE.
**Review ‍  If you are looking for some specific type of dish, then download the Chefz app and browse through all the restaurants and menus to check different cuisines. It is the finest restaurant finder app in UAE. ‍ ​
-------
Sentence r_id:  10249 equals review r_id:  10249
*Sentence It is the finest restaurant finder app in UAE.
**Review  ‌If you are looking for some specific type of dish, then download the Chefz app and browse through all the restaurants and menus to check different cuisines. It is the finest restaurant finder app in UAE. ​
-------
Sentence r_id:  10275 equals review r_id:  10275
*Sentence It is the finest restaurant finder app in UAE.
**Review    If you are looking for some specific type of dish, then download the Chefz app and browse through all the restaurants and menus to check different cuisines. It is the finest restaurant finder app in UAE. ​‌
-------
Se

In [886]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df = df[['r_id', 'reference_review']].drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
df = df.reset_index(drop=True)

In [887]:
len(df)

982

In [888]:
df.to_csv('reference_reviews.csv', index = False)
# This file will be used for sentiStrength

# Keep collocations match the patterns (noun noun, adjective noun, or verb noun)

In [388]:
data = pd.read_csv('collocations.csv') 
df = pd.DataFrame(data, columns=['r_id', 's_id', 'app_name','app_id', 'date', 'title', 'review', 'pos', 'tagged_sentences', 'new_tagged_sentences', 'collocations', 'rating'])

In [389]:
collocations_list = df['collocations'].tolist()
pos_list = df['pos'].tolist()

In [390]:
for x in range(len(collocations_list)):
    collocations_list[x] = literal_eval(collocations_list[x])

In [391]:
for x in range(len(pos_list)):
    pos_list[x] = literal_eval(pos_list[x])

In [446]:
new_list = []
for x in range(len(collocations_list)):
    new_list.insert(x, [])
    for y in range(len(collocations_list[x])):
        w1, w2 = collocations_list[x][y]
        for z in range(len(pos_list[x])):
            token, tag = pos_list[x][z]
            if re.match(token, w2):
                print("match word", w2, token)
                if re.match(r'n', tag):
                    print("----- Good pattern", w1, w2)
                    print("X =", x)
                    new_list[x].append((w1, w2))
                    break;
                else:
                    print("Unwanted tage", tag)

match word app app
----- Good pattern decent app
X = 0
match word job job
----- Good pattern do job
X = 1
match word do do
Unwanted tage v
match word many many
Unwanted tage a
match word area area
----- Good pattern deliver area
X = 2
match word off off
Unwanted tage v
match word apps apps
----- Good pattern many apps
X = 2
match word deliver deliver
Unwanted tage v
match word cash cash
----- Good pattern accept cash
X = 3
match word do do
Unwanted tage v
match word cold cold
Unwanted tage a
match word delivery delivery
----- Good pattern cash delivery
X = 3
match word take take
Unwanted tage v
match word get get
Unwanted tage v
match word little little
Unwanted tage a
match word bit bit
----- Good pattern little bit
X = 3
match word time time
----- Good pattern lot time
X = 3
match word food food
----- Good pattern sometimes food
X = 3
match word happy happy
Unwanted tage a
match word upsetting upsetting
Unwanted tage a
match word i i
----- Good pattern card i
X = 4
match word card ca

Unwanted tage v
match word tell tell
Unwanted tage v
match word anything anything
----- Good pattern tell anything
X = 83
match word driver driver
----- Good pattern assign driver
X = 84
match word food food
----- Good pattern bad food
X = 84
match word location location
----- Good pattern care location
X = 84
match word customer customer
----- Good pattern communicate customer
X = 84
match word care care
----- Good pattern customer care
X = 84
match word application application
----- Good pattern delivery application
X = 84
match word everyone everyone
----- Good pattern driver everyone
X = 84
match word delivery delivery
----- Good pattern food delivery
X = 84
match word way way
----- Good pattern proper way
X = 84
match word order order
----- Good pattern restaurant order
X = 84
match word say say
Unwanted tage v
match word service service
----- Good pattern say service
X = 85
match word wadi wadi
----- Good pattern service wadi
X = 85
match word dawasir dawasir
----- Good pattern w

match word anyone anyone
----- Good pattern app anyone
X = 173
match word twitter twitter
----- Good pattern app twitter
X = 173
match word response response
----- Good pattern be response
X = 173
match word support support
----- Good pattern contact support
X = 173
match word recommend recommend
Unwanted tage v
match word dont dont
Unwanted tage v
match word i i
Unwanted tage v
match word team team
----- Good pattern support team
X = 173
match word contact contact
Unwanted tage v
match word be be
Unwanted tage v
match word face face
Unwanted tage v
match word experience experience
----- Good pattern app experience
X = 174
match word app app
Unwanted tage a
match word wish wish
Unwanted tage v
match word i i
----- Good pattern experience i
X = 174
match word i i
----- Good pattern have i
X = 174
match word do do
Unwanted tage v
match word have have
Unwanted tage v
match word have have
Unwanted tage v
match word anyone anyone
----- Good pattern wish anyone
X = 174
match word location lo

Unwanted tage a
match word next next
Unwanted tage a
match word support support
----- Good pattern contact support
X = 251
match word good good
Unwanted tage a
match word job job
----- Good pattern good job
X = 251
match word contact contact
Unwanted tage v
match word be be
Unwanted tage v
match word careful careful
Unwanted tage a
match word time time
----- Good pattern next time
X = 251
match word tell tell
Unwanted tage v
match word app app
----- Good pattern delete app
X = 252
match word delete delete
Unwanted tage v
match word unlinked unlinked
Unwanted tage a
match word device device
----- Good pattern unlinked device
X = 252
match word service service
----- Good pattern costumer service
X = 253
match word trash trash
Unwanted tage a
match word costumer costumer
----- Good pattern trash costumer
X = 253
match word money money
----- Good pattern waste money
X = 253
match word easy easy
----- Good pattern app easy
X = 254
match word app app
----- Good pattern awesome app
X = 254
ma

----- Good pattern sm minute
X = 351
match word couple couple
----- Good pattern try couple
X = 351
match word yesterday yesterday
----- Good pattern anything yesterday
X = 352
match word do do
Unwanted tage v
match word sad sad
Unwanted tage a
match word support support
Unwanted tage v
match word order order
----- Good pattern i order
X = 352
match word i i
----- Good pattern io i
X = 352
match word anything anything
----- Good pattern order anything
X = 352
match word app app
----- Good pattern sad app
X = 352
match word io io
----- Good pattern support io
X = 352
match word be be
Unwanted tage v
match word other other
Unwanted tage a
match word want want
Unwanted tage v
match word delivery delivery
----- Good pattern free delivery
X = 353
match word increase i
----- Good pattern have increase
X = 353
match word notice notice
Unwanted tage v
match word price price
----- Good pattern increase price
X = 353
match word application application
----- Good pattern junk application
X = 353


...

match word be be
Unwanted tage v
match word i i
----- Good pattern app i
X = 3264
match word best be
Unwanted tage v
match word best best
Unwanted tage a
match word app app
----- Good pattern best app
X = 3264
match word app app
----- Good pattern chefz app
X = 3264
match word come come
Unwanted tage v
match word have have
Unwanted tage v
match word friendly friendly
Unwanted tage a
match word staff staff
----- Good pattern delivery staff
X = 3265
match word knowledgeable knowledgeable
Unwanted tage a
match word educate educate
Unwanted tage v
match word be be
Unwanted tage v
match word satisfied satisfied
Unwanted tage a
match word be be
Unwanted tage v
match word produce produce
Unwanted tage v
match word software software
----- Good pattern satisfied software
X = 3266
match word people people
----- Good pattern software people
X = 3266
match word be be
Unwanted tage v
match word i i
----- Good pattern app i
X = 3267
match word best be
Unwanted tage v
match word best best
Unwanted ta

match word app app
----- Good pattern delivery app
X = 3387
match word delivery delivery
----- Good pattern food delivery
X = 3387
match word food food
----- Good pattern have food
X = 3387
match word order order
----- Good pattern effortless order
X = 3388
match word love love
Unwanted tage v
match word effortless effortless
----- Good pattern interface effortless
X = 3388
match word interface i
----- Good pattern love interface
X = 3388
match word food food
----- Good pattern order food
X = 3388
match word fast fast
Unwanted tage a
match word app app
----- Good pattern best app
X = 3389
match word delivery delivery
----- Good pattern fast delivery
X = 3389
match word range range
----- Good pattern broad range
X = 3390
match word wide wide
Unwanted tage a
match word broad broad
Unwanted tage a
match word searchable searchable
Unwanted tage a
match word restaurant restaurant
----- Good pattern range restaurant
X = 3390
match word choice choice
----- Good pattern restaurant choice
X = 3

----- Good pattern include lot
X = 3507
match word option option
----- Good pattern lot option
X = 3507
match word easy easy
Unwanted tage a
match word strong strong
Unwanted tage a
match word support support
----- Good pattern delivery support
X = 3508
match word be be
Unwanted tage v
match word bookmark bookmark
Unwanted tage v
match word allow allow
Unwanted tage v
match word favorite favorite
Unwanted tage a
match word doorstep doorstep
----- Good pattern deliver doorstep
X = 3509
match word love love
----- Good pattern fall love
X = 3509
match word food food
----- Good pattern favorite food
X = 3509
match word get get
Unwanted tage v
match word deliver deliver
Unwanted tage v
match word app app
----- Good pattern love app
X = 3509
match word foodie foodie
----- Good pattern be foodie
X = 3510
match word time time
----- Good pattern cuisine time
X = 3510
match word i i
----- Good pattern foodie i
X = 3510
match word be be
Unwanted tage v
match word love love
Unwanted tage v
match w

match word range range
----- Good pattern wide range
X = 3632
match word payment payment
----- Good pattern make payment
X = 3633
match word quick quick
Unwanted tage a
match word approve approve
Unwanted tage v
match word food food
----- Good pattern brilliant food
X = 3635
match word mobile mobile
----- Good pattern delivery mobile
X = 3635
match word praise praise
----- Good pattern deserve praise
X = 3635
match word brilliant brilliant
Unwanted tage a
match word deserve deserve
Unwanted tage v
match word delivery delivery
----- Good pattern food delivery
X = 3635
match word app app
----- Good pattern mobile app
X = 3635
match word develop develop
Unwanted tage v
match word united united
Unwanted tage a
match word emirate emirate
----- Good pattern arab emirate
X = 3636
match word look look
Unwanted tage v
match word food food
----- Good pattern best food
X = 3636
match word search search
----- Good pattern emirate search
X = 3636
match word chefz chefz
----- Good pattern end chefz


----- Good pattern online chefz
X = 3749
match word finder finder
----- Good pattern restaurant finder
X = 3749
match word enjoy enjoy
Unwanted tage v
match word app app
----- Good pattern chefz app
X = 3750
match word tastiest tastiest
----- Good pattern classiest tastiest
X = 3750
match word restaurant restaurant
----- Good pattern elite restaurant
X = 3750
match word classiest classiest
----- Good pattern enjoy classiest
X = 3750
match word top top
Unwanted tage a
match word chefz chefz
----- Good pattern restaurant chefz
X = 3750
match word meal meal
----- Good pattern tastiest meal
X = 3750
match word elite elite
Unwanted tage a
match word food food
----- Good pattern amaze food
X = 3751
match word say say
Unwanted tage v
match word be be
Unwanted tage v
match word i i
----- Good pattern more i
X = 3752
match word less less
Unwanted tage a
match word star star
----- Good pattern app star
X = 3753
match word give give
Unwanted tage v
match word qulaity qulaity
----- Good pattern be

In [474]:
table = {"r_id": [], "s_id": [], "app_name": [], "app_id": [], "date": [], "title": [], "review": [], "pos": [], "tagged_sentences": [], "collocations": [], "rating": [], "new_tagged_sentences": [], "collocations_follow_pattern": []}

# store each collocation in a separate row
for x in range(len(df)):
    for y in range(len(new_list[x])):
        table["r_id"].append(df['r_id'][x])
        table["s_id"].append(df['s_id'][x])
        table["app_name"].append(df['app_name'][x])
        table["app_id"].append(df['app_id'][x])
        table["date"].append(df['date'][x])
        table["title"].append(df['title'][x])
        table["review"].append(df['review'][x])
        table["pos"].append(df['pos'][x])
        table["tagged_sentences"].append(df['tagged_sentences'][x])
        table["rating"].append(df['rating'][x])
        table["new_tagged_sentences"].append(df['new_tagged_sentences'][x])
        table["collocations"].append(df['collocations'][x])
        table["collocations_follow_pattern"].append(new_list[x][y])

In [475]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('collocations_follow_pattern_splitted.csv', index = False)